На домашнем ПК ОЗУ не справляется с несколькими миллионами строк, поэтому в работе пришлось использовать датасеты меньшего размера :( 

## Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [2]:
df = pd.read_table(r".\ml-100k\u.data", engine='python', header=0,
                   names=['UserID','MovieID','Rating','Timestamp'])
df.head(2)

,UserID,MovieID,Rating,Timestamp
0,186,302,3,891717742
1,22,377,1,878887116


In [3]:
df['class'] = df['Rating'].apply(lambda x: 'low' if x <= 2 else
                                             ('middle' if x <= 4 else
                                                ('high' if x == 4.5 or x == 5 else None)))
df.head(2)

,UserID,MovieID,Rating,Timestamp,class
0,186,302,3,891717742,middle
1,22,377,1,878887116,low


In [4]:
df['class'] .value_counts()

middle    61318
high      21201
low       17480
Name: class, dtype: int64

## Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [5]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск'] }

In [6]:
df1 = pd.read_csv(r"keywords.csv", sep=',')
df1.head(2)

,keyword,shows
0,вк,64292779
1,одноклассники,63810309


In [7]:
def get_region(x):
    for region, cities in geo_data.items():
        for city in cities:
            if city in x.lower(): return region
    return 'undefined'

In [8]:
df1['region'] = df1['keyword'].apply(get_region)
df1['region'].value_counts()

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

## Задание 3 (бонусное)
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [9]:
ratings_df = pd.read_table(r".\ml-latest-small\ratings.csv", sep=',', header=0)
ratings_df.head(2)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247


In [10]:
movies_df = pd.read_csv(r".\ml-latest-small\movies.csv", sep=',', header=0)
movies_df.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [11]:
df2 = pd.merge(ratings_df, movies_df, how = 'inner')
df2.head(2)

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [12]:
years = range(1950,2011)

def production_year(x):
    for year in years:
        if str(year) in x: return year
    return 1900

In [13]:
df2['year'] = df2['title'].apply(production_year)

In [14]:
df2.groupby('year').agg({'rating':'mean'}).sort_values('rating', ascending=False)

,rating
year,
1957,4.039535
1954,4.009191
1962,3.969466
1952,3.953125
1972,3.944293
...,...
2000,3.398922
2005,3.359976
1992,3.353555
